# Recognize named entities on Twitter with LSTMs
starter code https://github.com/hse-aml/natural-language-processing/blob/master/week2/week2-NER.ipynb

In [1]:
import numpy as np
import tensorflow as tf
from evaluation import precision_recall_f1

In [2]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token

            if(token.startswith('@')):
                token = '<USR>'
                
            elif(token.startswith('http://') or token.startswith('https://')):
                token = '<URL>'
                
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [3]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [4]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



In [5]:
from collections import defaultdict
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0
    
    idx = 0
    for token in special_tokens:
        tok2idx[token] = idx
        idx2tok.append(token)
        idx += 1

    for token_list in tokens_or_tags:
        for token in token_list:
            if token not in tok2idx:
                tok2idx[token] = idx
                idx2tok.append(token)
                idx += 1
    
    return tok2idx, idx2tok

In [6]:
special_tokens  = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [27]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

# Build Bi-LSTM

In [33]:
class BiLSTMModel():
    def __init__(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
        self.create_placeholders()
        self.build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
        self.compute_predictions()
        self.compute_loss(n_tags, PAD_index)
        self.create_optimizer()
        
    def create_placeholders(self):
        
        # Placeholders for input and ground truth output.
        self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
        self.ground_truth_tags = self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')

        # Placeholder for lengths of the sequences.
        self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 

        # Placeholder for a dropout keep probability. If we don't feed
        # a value for this placeholder, it will be equal to 1.0.
        self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])

        # Placeholder for a learning rate (tf.float32).
        self.learning_rate_ph = tf.placeholder_with_default(1e4, shape=[])
    
    def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
        # Create embedding variable (tf.Variable) with dtype tf.float32
        initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
        embedding_matrix_variable = tf.Variable(initial_embedding_matrix, name='embeddings_matrix', dtype=tf.float32)
        embeddings = tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)
  

        forward_cell = tf.nn.rnn_cell.DropoutWrapper(
            tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
            input_keep_prob=self.dropout_ph,
            output_keep_prob=self.dropout_ph,
            state_keep_prob=self.dropout_ph
        )
        backward_cell = tf.nn.rnn_cell.DropoutWrapper(
            tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
            input_keep_prob=self.dropout_ph,
            output_keep_prob=self.dropout_ph,
            state_keep_prob=self.dropout_ph
        )
        
        (rnn_output_fw, rnn_output_bw), _ = tf.nn.bidirectional_dynamic_rnn(
        cell_fw= forward_cell, cell_bw= backward_cell,
        dtype=tf.float32,
        inputs=embeddings,
        sequence_length=self.lengths
        )
        rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)
        self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)
    
    def compute_loss(self, n_tags, PAD_index):
        ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
        loss_tensor = tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth_tags_one_hot, logits=self.logits)

        # Create loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
        mask = tf.cast(tf.not_equal(loss_tensor, PAD_index), tf.float32)
        self.loss =  tf.reduce_mean(tf.reduce_sum(tf.multiply(loss_tensor, mask), axis=-1) / tf.reduce_sum(mask, axis=-1))
        
    def create_optimizer(self):
        # Create an optimizer (tf.train.AdamOptimizer)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate_ph)
        self.grads_and_vars = self.optimizer.compute_gradients(self.loss)

        # Gradient clipping (tf.clip_by_norm) for self.grads_and_vars
        # Pay attention that you need to apply this operation only for gradients 
        # because self.grads_and_vars contains also variables.
        # list comprehension might be useful in this case.
        clip_norm = tf.cast(1.0, tf.float32)
        self.grads_and_vars = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in self.grads_and_vars]

        self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)
    
    def compute_predictions(self):
        # Create softmax (tf.nn.softmax) function
        softmax_output = tf.nn.softmax(self.logits)

        # Use argmax (tf.argmax) to get the most probable tags
        # Don't forget to set axis=-1
        # otherwise argmax will be calculated in a wrong way
        self.predictions = tf.argmax(softmax_output, axis=-1)
    
    def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
        feed_dict = {self.input_batch: x_batch,
                     self.ground_truth_tags: y_batch,
                     self.learning_rate_ph: learning_rate,
                     self.dropout_ph: dropout_keep_probability,
                     self.lengths: lengths}

        _, loss = session.run((self.train_op, self.loss), feed_dict=feed_dict)
        return loss
    
    def predict_for_batch(self, session, x_batch, lengths):
        predictions = session.run(self.predictions, feed_dict={self.input_batch:x_batch, self.lengths:lengths})
        return predictions
        
    

# Batch generator

In [37]:
def batch_generator(batch_size, tokens, tags, shuffle=True, allow_small_last_batch=True):
    num_samples = len(tokens)
    if shuffle:
        sample_idxs = np.random.permutation(num_samples)
    else:
        sample_idxs = np.arange(num_samples)
        
    num_batches = num_samples//batch_size
    
    if allow_small_last_batch and num_samples%batch_size:
        num_batches += 1
    
    for k in range(num_batches):
        batch_start = k * batch_size
        batch_end = min((k+1) * batch_size, num_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        
        for idx in sample_idxs[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
        
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            l = len(x_list[n])
            x[n, :l] = x_list[n]
            y[n, :l] = y_list[n]
            lengths[n] = l
        yield x, y, lengths

## Evaluation

In [39]:
def predict_tags (model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batch_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

# Train and evaluate model

In [40]:
tf.reset_default_graph()
model = BiLSTMModel(20505, 21, 200, 200, token2idx['<PAD>'])

batch_size = 32
num_epochs = 4
learning_rate = 0.005
learning_rate_decay = 1.414
dropout_keep_prob = 0.5

sess = tf.Session()
sess.run(tf.global_variables_initializer())

step = 0
for epoch in range(num_epochs):
    for x_batch, y_batch, lengths in batch_generator(batch_size, train_tokens, train_tags):
        
        step += 1
        loss = model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_prob)

    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(num_epochs) + '-' * 20)
    print('Training loss: ', loss)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    learning_rate = learning_rate/learning_rate_decay
    
print('...training finished.')

-------------------- Epoch 1 of 4 --------------------
Training loss:  1.0196917
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 2573 phrases; correct: 433.

precision:  16.83%; recall:  9.65%; F1:  12.26

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 230 phrases; correct: 33.

precision:  14.35%; recall:  6.15%; F1:  8.60

-------------------- Epoch 2 of 4 --------------------
Training loss:  0.25636837
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4540 phrases; correct: 1948.

precision:  42.91%; recall:  43.39%; F1:  43.15

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 336 phrases; correct: 136.

precision:  40.48%; recall:  25.33%; F1:  31.16

-------------------- Epoch 3 of 4 --------------------
Training loss:  0.27773154
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4659 phrases; correct: 2816.

precision:  60.44%; recall:  62.73%; F1:  61.57